In [ ]:
library(tidyverse)
library(RColorBrewer)

In [ ]:
# 1) count unmapped from unmapped file 
# 2) take soft clipped kmer numbers -> S in CIGAR
# 3) insertions -> I in CIGAR
# 4) deletions -> D in CIGAR

In [ ]:
paramspace <- read.csv("../data/metadata/paramspace_phaseI.csv")
paramspace <- paramspace %>% select(pt_id, output_path_unique_kmers)
head(paramspace)
dim(paramspace)

In [ ]:
create_fasta_unmapped_kmers <- function(dataset, pt, folder) {
    file.create(paste0("../data/", pt, "/", folder, "/all_germline_filtered_bams_tumor_ci5_cs1e9/mapping_to_reference/unmapped_kmers.fasta"))

    for (row in 1:nrow(dataset)){
        cat(paste0(">", dataset[row, "V1"]), file=paste0("../data/", pt, "/", folder, "/all_germline_filtered_bams_tumor_ci5_cs1e9/mapping_to_reference/unmapped_kmers.fasta"),append=TRUE, sep="\n")
        cat(dataset[row, "V10"], file=paste0("../data/", pt, "/", folder, "/all_germline_filtered_bams_tumor_ci5_cs1e9/mapping_to_reference/unmapped_kmers.fasta"),append=TRUE, sep="\n")
    }
    
}

In [ ]:
res_unmapped_final <- NULL
for (row in 1:nrow(paramspace)){
    patient = paramspace[row, "pt_id"]
    folder = paramspace[row, "output_path_unique_kmers"]
    print(patient)
    
    res_unmapped <- read.table(paste0('../data/', patient, "/", folder, "/all_germline_filtered_bams_tumor_ci5_cs1e9/mapping_to_reference/unique_tumor_kmers_unmapped.sam"))
    print(nrow(res_unmapped))
    res_unmapped_pt <- tibble(pt_id = patient, tumor_folder = folder, unmapped_kmers = nrow(res_unmapped))
    res_unmapped_final <- rbind(res_unmapped_final, res_unmapped_pt)
    
    res_unmapped$V10 <- as.character(res_unmapped$V10)
    
    #create_fasta_unmapped_kmers(res_unmapped, patient, folder)
}

In [ ]:
head(res_unmapped)

In [ ]:
create_fasta <- function(dataset, pt, folder) {
    file.create(paste0("../data/", pt, "/", folder, "/all_germline_filtered_bams_tumor_ci5_cs1e9/mapping_to_reference/kmers_mapping_w_01_mismatches.fasta"))
    #fileConn<-file(paste0("../data/", pt, "/", folder, "/all_germline_filtered_bams_tumor_ci5_cs1e9/mapping_to_reference/kmers_mapping_w_01_mismatches.fasta"))
    for (row in 1:nrow(dataset)){
        cat(paste0(">", dataset[row, "V1"]), file=paste0("../data/", pt, "/", folder, "/all_germline_filtered_bams_tumor_ci5_cs1e9/mapping_to_reference/kmers_mapping_w_01_mismatches.fasta"),append=TRUE, sep="\n")
        cat(dataset[row, "V10"], file=paste0("../data/", pt, "/", folder, "/all_germline_filtered_bams_tumor_ci5_cs1e9/mapping_to_reference/kmers_mapping_w_01_mismatches.fasta"),append=TRUE, sep="\n")
    }
    
}

In [ ]:
res_mapped_final <- NULL
for (row in 1:nrow(paramspace)){
    patient = paramspace[row, "pt_id"]
    folder = paramspace[row, "output_path_unique_kmers"]
    print(as.character(patient))
    # , colClasses=c(rep("character", 15)) 
    res_mapped <- read.table(paste0('../data/', patient, "/", folder, "/all_germline_filtered_bams_tumor_ci5_cs1e9/mapping_to_reference/unique_tumor_kmers_mapped.sam"), header=FALSE, sep="\t", fill=TRUE,
                             col.names = paste0("V", seq_len(17)))
    
    #print(head(res_mapped))
    res_mapped_softclipped <- dplyr::filter(res_mapped, grepl('S|H', V6))
    res_mapped_rest <- dplyr::filter(res_mapped, !(grepl('S|H', V6)))
    
    res_mapped_indels <- dplyr::filter(res_mapped_rest, grepl('I|D', V6))
    res_mapped_rest <- dplyr::filter(res_mapped_rest, !(grepl('I|D', V6)))
    
    res_mapped_rest_with_indels_check <- dplyr::filter(res_mapped_rest, !(grepl('^', V13)))
    #print(dim(res_mapped_rest_with_indels_check))
    
    #print(length(res_mapped_rest$V1))
    #print(length(res_mapped_rest$V10))
    res_mapped_rest$V12 <- as.character(res_mapped_rest$V12)
    res_mapped_rest_split <- res_mapped_rest %>% separate(V12, c("A", "B", "C"), ":", extra = "merge")
    #print(head(res_mapped_rest_split))
    #print(dim(res_mapped_rest_split))
    print(dim(res_mapped_rest_split %>% filter(A == "NM")))
    res_mapped_rest_split$C <- as.numeric(res_mapped_rest_split$C) 

    res_mapped_pt <- tibble(pt_id = patient, 
                            tumor_folder = folder, 
                            mapped_kmers = nrow(res_mapped),
                            softclipped_kmers = nrow(res_mapped_softclipped),
                            kmers_w_indels = nrow(res_mapped_indels),
                            kmers_rest = nrow(res_mapped_rest),
                            kmers_0_mismatches = nrow(res_mapped_rest_split %>% filter(C == 0)),
                            kmers_1_mismatches = nrow(res_mapped_rest_split %>% filter(C == 1)),
                            kmers_2_mismatches = nrow(res_mapped_rest_split %>% filter(C == 2)),
                            kmers_3more_mismatches = nrow(res_mapped_rest_split %>% filter(C >= 3))
                            )
    
    print("Kmers with 0 or 1 mismatches")
    res_01_mismatches <- res_mapped_rest_split %>% filter(C <= 1)
    print(dim(res_01_mismatches))
    #print(head(res_01_mismatches))
    res_01_mismatches$V10 <- as.character(res_01_mismatches$V10)
    
    #print("duplicated kmers: ")
    #print(res_01_mismatches$V10[duplicated(res_01_mismatches$V10)])
    #n_occur <- data.frame(table(res_01_mismatches$V10))
    #print(n_occur[n_occur$Freq > 1,])
    #print(res_01_mismatches[res_01_mismatches$V10 %in% n_occur$Var1[n_occur$Freq > 1],])
    
    #for (i in res_01_mismatches$V10){
    #    if (nchar(i) != 50){
    #        print(i)
    #    }
    #}
    
    #write.table(res_01_mismatches, file = paste0("../data/", patient, "/", folder, "/all_germline_filtered_bams_tumor_ci5_cs1e9/mapping_to_reference/kmers_mapping_w_01_mismatches.tsv"), row.names=FALSE, sep="\t")
    
    res_mapped_final <- rbind(res_mapped_final, res_mapped_pt)
}

print(head(res_mapped))

In [ ]:
dim(res_unmapped_final)
head(res_unmapped_final)

In [ ]:
dim(res_mapped_final)
head(res_mapped_final)

In [ ]:
low_qual_sample <- read.table("../low_qual_sample.txt")
low_qual_sample <- as.character(low_qual_sample[[1]])

phaseIpt_R <- read.csv("../phaseI_pt_R.csv", header=FALSE)
phaseIpt_R <- as.character(unlist(c(phaseIpt_R[1,])))

In [ ]:
res_mapped_final_nopt <- res_mapped_final %>% filter(!(pt_id == low_qual_sample))

In [ ]:
a <- ifelse(res_mapped_final$pt_id %in% phaseIpt_R, "red", "darkblue")
b <- ifelse(unique(as.character(res_mapped_final_nopt$pt_id)) %in% phaseIpt_R, "red", "darkblue")

In [ ]:
res <- left_join(res_unmapped_final, res_mapped_final, by = c("pt_id", "tumor_folder"))
head(res)

In [ ]:
res_long <- res %>% pivot_longer(cols = c("unmapped_kmers", 
                                          "mapped_kmers", 
                                          "softclipped_kmers", 
                                          "kmers_w_indels",
                                          "kmers_rest",
                                          "kmers_0_mismatches",
                                          "kmers_1_mismatches",
                                          "kmers_2_mismatches",
                                          "kmers_3more_mismatches"),
                                                 names_to = "kmer",
                                                 values_to = "n")

In [ ]:
head(res_long)

In [ ]:
options(repr.plot.width=15, repr.plot.height=8)

res_long %>% filter(kmer %in% c("mapped_kmers", 
                                "unmapped_kmers")) %>% 
    ggplot() +
    geom_col(aes(x = pt_id, y = n, fill = kmer), position = "dodge2") +
    ggtitle("Mapped and unmapped unique tumor kmers") + theme_minimal() + 
    theme(axis.text.x = element_text(colour = a))

In [ ]:
res_long %>% 
    filter(!(pt_id %in% low_qual_sample)) %>% 
    filter(kmer %in% c("mapped_kmers", 
                       "unmapped_kmers")) %>% 
    ggplot() +
    geom_col(aes(x = pt_id, y = n, fill = kmer), position = "dodge2") + theme_minimal() +
    theme(axis.text.x = element_text(colour = b, size = 14),
          axis.text.y = element_text(size = 14), 
          axis.title.y = element_text(size = 16),
          legend.title=element_text(size=16), 
          legend.text=element_text(size=14)) +
    scale_fill_discrete(name = "K-mers", labels = c("Mapped", "Unmapped")) + xlab("") + ylab("Count")
    


In [ ]:
res_long %>% 
    filter(!(kmer %in% c("mapped_kmers", 
                         "unmapped_kmers",
                         "kmers_0_mismatches",
                         "kmers_1_mismatches",
                         "kmers_2_mismatches",
                         "kmers_3more_mismatches"))) %>% 
    ggplot() +
    geom_col(aes(x = pt_id, y = n, fill = kmer), position = "dodge2") +
    ggtitle("Mapped unique tumor kmers") + theme_minimal() + 
    theme(axis.text.x = element_text(colour = a))

In [ ]:
res_long %>% 
    filter(!(pt_id %in% low_qual_sample)) %>% 
    filter(!(kmer %in% c("mapped_kmers", 
                         "unmapped_kmers",
                         "kmers_rest",
                         "kmers_0_mismatches",
                         "kmers_1_mismatches",
                         "kmers_2_mismatches",
                         "kmers_3more_mismatches"))) %>% 
    ggplot() +
    geom_col(aes(x = pt_id, y = n, fill = kmer), position = "dodge2") +
    theme_minimal() + 
    theme(axis.text.x = element_text(colour = b, size = 14),
          axis.text.y = element_text(size = 14), 
          axis.title.y = element_text(size = 16),
          legend.title=element_text(size=16), 
          legend.text=element_text(size=14)) +
    scale_fill_discrete(name = "K-mers", labels = c("including indels", "bases clipped")) + xlab("") + ylab("Count")



In [ ]:
res_long %>% 
    filter(!(kmer %in% c("mapped_kmers", 
                         "unmapped_kmers",
                         "softclipped_kmers", 
                         "kmers_w_indels",
                         "kmers_rest"))) %>% 
    ggplot() +
    geom_col(aes(x = pt_id, y = n, fill = kmer), position = "dodge2") +
    ggtitle("Unique tumor kmers, mapped with 0, 1, 2, 3 or more mismatches") + theme_minimal() + 
    theme(axis.text.x = element_text(colour = a))

In [ ]:
res_long %>% 
    filter(!(pt_id %in% low_qual_sample)) %>% 
    filter(!(kmer %in% c("mapped_kmers", 
                         "unmapped_kmers",
                         "softclipped_kmers", 
                         "kmers_w_indels",
                         "kmers_rest"))) %>% 
    ggplot() +
    geom_col(aes(x = pt_id, y = n, fill = kmer), position = "dodge2") +
    theme_minimal() + 
     theme(axis.text.x = element_text(colour = b, size = 14),
          axis.text.y = element_text(size = 14), 
          axis.title.y = element_text(size = 16),
          legend.title=element_text(size=16), 
          legend.text=element_text(size=14)) +
    scale_fill_discrete(name = "K-mers", labels = c("0 mismatches", "1 mismatch", "2 mismatches", "3 or more mismatches")) + xlab("") + ylab("log10(Count)")




In [ ]:
str(res_long)

In [ ]:
res_long %>% filter(kmer %in% c("unmapped_kmers", "mapped_kmers"))

In [ ]:
all_kmers <- res_long %>% group_by(pt_id) %>% filter(kmer %in% c("mapped_kmers", "unmapped_kmers")) %>% 
    summarize(all_kmers_n = sum(n))
(all_kmers)

In [ ]:
unmapped <- res_long %>% group_by(pt_id) %>% filter(kmer %in% c("unmapped_kmers")) %>% rename(unmapped_kmers_n = n)
head(unmapped)

In [ ]:
unmapped_ratio <- left_join(unmapped, all_kmers, by = "pt_id") 
unmapped_ratio <- unmapped_ratio %>% mutate(unmapped_ratio = unmapped_kmers_n/all_kmers_n) %>% arrange(desc(unmapped_ratio))
unmapped_ratio